# 🏋️‍♂️ Assistente per la Salute e il Benessere 🌱  

## Descrizione  
Questo assistente AI ti aiuta a migliorare il tuo benessere raccogliendo informazioni su di te e fornendo consigli personalizzati su **nutrizione** ed **esercizio fisico**.  

## Funzionalità  
- 📊 **Calcolo del BMI**: Analizza il tuo indice di massa corporea e fornisce una classificazione.  
- 🏃 **Suggerimenti per l'allenamento**: In base ai tuoi obiettivi e livello di attività.  
- 🍏 **Consigli nutrizionali**: Linee guida alimentari per supportare il tuo percorso di benessere.  
- 📄 **Report riepilogativo**: Genera un documento con il tuo stato di salute e suggerimenti personalizzati.  

## Come funziona  
1. L'assistente raccoglie i tuoi dati in modo naturale e interattivo.  
2. Esegue calcoli e genera suggerimenti in base alle informazioni fornite.  
3. Ti guida passo dopo passo nel raggiungere i tuoi obiettivi di fitness e salute.  

> ⚠️ *Nota*: L'assistente non fornisce consigli medici. Consulta un professionista per indicazioni personalizzate.  


In [15]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [16]:
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [27]:
###############################################################################
# System Prompt
###############################################################################
system_message = """Sei un assistente per la salute e il benessere. Il tuo compito è raccogliere informazioni dagli utenti attraverso domande mirate e fornire suggerimenti personalizzati 
su esercizi e nutrizione in base ai loro obiettivi. 

Durante l'onboarding, raccogli le informazioni una alla volta, ponendo una domanda per ogni dato. Chiedi prima il nome, poi usalo per rivolgere all’utente le domande successive, così da apparire più naturale e umano. 
Le informazioni da raccogliere sono: 
- Nome 
- Età 
- Genere 
- Altezza (in cm) 
- Peso (in kg) 

Dopo aver raccolto tutti i dati, calcola il BMI (Body Mass Index), chiamando la funzione calculate_bmi, e restituisci il risultato all'utente.

Fai in modo che il chatbot esegua uno step per volta: prima chiede il nome, poi lo usa per formulare la domanda successiva, e così via fino a raccogliere tutte le informazioni necessarie.

Rispondi sempre in modo chiaro, conciso e motivante, incoraggiando l'utente nel suo percorso di benessere. 
Evita consigli medici specifici e raccomanda di consultare un professionista della salute per piani personalizzati più approfonditi."""


###############################################################################
#- Livello di attività fisica (considerando anche il lavoro attuale)
#- Preferenze alimentari 
#- Obiettivo (es. perdere peso, aumentare la massa muscolare, mantenere la forma fisica) 
#- Orizzonte temporale per il raggiungimento dell'obiettivo
#
#Dopo aver raccolto tutti i dati, calcola il BMI (Body Mass Index) e classificalo secondo le categorie standard (sottopeso, normopeso, sovrappeso, obesità). 
#
#Usa queste informazioni per suggerire un programma di esercizi e una linea guida nutrizionale adeguata.
#
#Infine, genera un report riepilogativo con:
#- Il BMI calcolato e la sua classificazione 
#- Il piano di allenamento suggerito 
#- Le linee guida nutrizionali 
#
#Il report deve essere chiaro e scaricabile come file di testo. Se l'utente richiede ulteriori dettagli o modifiche al piano, aggiorna le informazioni di conseguenza.
#
#Fai in modo che il chatbot esegua uno step per volta: prima chiede il nome, poi lo usa per formulare la domanda successiva, e così via fino a raccogliere tutte le informazioni necessarie.
#
#Rispondi sempre in modo chiaro, conciso e motivante, incoraggiando l'utente nel suo percorso di benessere. Evita consigli medici specifici e raccomanda di consultare un professionista della salute per piani personalizzati più approfonditi."""


In [28]:
###############################################################################
# Helper Functions
###############################################################################

def calculate_bmi(height, weight):

    if height and weight:
        bmi = weight / ((height / 100) ** 2)
        if bmi < 18.5:
            status = "Sottopeso"
        elif 18.5 <= bmi < 24.9:
            status = "Normopeso"
        elif 25 <= bmi < 29.9:
            status = "Sovrappeso"
        else:
            status = "Obesità"
        return f"Il tuo BMI è {bmi:.2f}, che è considerato {status}."
    else:
        return "Mancano informazioni per calcolare il BMI."

def suggest_exercise(self):
    activity_level = self.user_profile.get("activity_level", "moderate")
    goal = self.user_profile.get("goal", "fitness")
    
    if goal == "gain muscle":
        return "Focus on strength training with progressive overload and high-protein intake."
    elif goal == "lose weight":
        return "Incorporate a mix of cardio and resistance training, and maintain a caloric deficit."
    elif goal == "maintain fitness":
        return "Stick to a balanced routine of strength training and moderate cardio."
    else:
        return "Please provide a goal to get personalized exercise suggestions."

def suggest_nutrition(self):
    dietary_preferences = self.user_profile.get("dietary_preferences", "balanced")
    goal = self.user_profile.get("goal", "fitness")
    
    if goal == "gain muscle":
        return "Increase protein intake with lean meats, eggs, legumes, and consider healthy fats."
    elif goal == "lose weight":
        return "Focus on portion control, high-fiber foods, and lean proteins while reducing processed foods."
    elif goal == "maintain fitness":
        return "Eat a well-balanced diet with a mix of macronutrients in appropriate proportions."
    else:
        return "Maintain a balanced diet with fruits, vegetables, whole grains, and lean proteins."

def generate_report(self):
    bmi_info = self.calculate_bmi()
    exercise_suggestion = self.suggest_exercise()
    nutrition_suggestion = self.suggest_nutrition()
    report = f"Health & Wellness Report:\n{bmi_info}\nExercise Plan: {exercise_suggestion}\nNutritional Guidance: {nutrition_suggestion}\n"
    
    with open("health_wellness_report.txt", "w") as file:
        file.write(report)
    
    return report
    
# Example usage:
#assistant = HealthWellnessAssistant()
#assistant.set_user_profile(age=30, gender="male", height=175, weight=70, activity_level="moderate", dietary_preferences="balanced", goal="gain muscle", timeframe="3 months")
#print(assistant.generate_report())


In [34]:
###############################################################################
# Tools JSON Schemas
###############################################################################
bmi_function = {
    "name": "calculate_bmi",
    "description": "Calculate BMI based on height and weight.",
    "parameters": {
        "type": "object",
        "properties": {
            "height": {
                "type": "number",
                "description": "height in cm",
            },
            "weight": {
                "type": "number",
                "description": "weight in kg",
            },
        },
        "required": ["height", "weight"],
    },
}


In [41]:
###############################################################################
# Handle Tool Calls
###############################################################################

def handle_tool_call(message):
    """
    The LLM can request to call a function in 'tools'. We parse the JSON arguments
    and run the Python function. Then we return a 'tool' message with the result.
    """
    tool_call = message.tool_calls[0]
    fn_name   = tool_call.function.name
    args      = json.loads(tool_call.function.arguments)

    if fn_name == "calculate_bmi":
        height = args.get("height")
        weight = args.get("weight")
        bmi = calculate_bmi(height, weight)

        response_content = {
            "height": height,
            "weight": weight,
            "bmi": bmi
        }
        print(response_content)

    else:
        response_content = {"error": f"Unknown tool: {fn_name}"}

    return {
        "role": "tool",
        "content": json.dumps(response_content),
        "tool_call_id": tool_call.id,
    }, args

In [36]:
###############################################################################
# Available Tools
###############################################################################
tools = [
    {"type": "function", "function": bmi_function},
]


In [37]:
###############################################################################
# Main Chat Function
###############################################################################
def chat(message, history):
    """
    The main chat loop that handles the conversation with the user,
    passing 'tools' definitions to the LLM for function calling.
    """
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    try:
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools
        )

        # If the LLM requests a function call, handle it
        while response.choices[0].finish_reason == "tool_calls":
            msg = response.choices[0].message
            print(f"[INFO] Tool call requested: {msg.tool_calls[0]}")
            tool_response, tool_args = handle_tool_call(msg)
            print(f"[INFO] Tool response: {tool_response}")

            # Add both the LLM's request and our tool response to the conversation
            messages.append(msg)
            messages.append(tool_response)

            # Re-send updated conversation to get final or next step
            response = openai.chat.completions.create(
                model=MODEL,
                messages=messages
            )

        # Return normal text response (finish_reason = "stop")
        return response.choices[0].message.content

    except Exception as e:
        print(f"[ERROR] {e}")
        return "I'm sorry, something went wrong while processing your request."

In [42]:
###############################################################################
# Launch Gradio
###############################################################################
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


[INFO] Tool call requested: ChatCompletionMessageToolCall(id='call_11y8VnXgKnehg0rTWk4v4ZYa', function=Function(arguments='{"height":187,"weight":99}', name='calculate_bmi'), type='function')
{'height': 187, 'weight': 99, 'bmi': 'Il tuo BMI è 28.31, che è considerato Sovrappeso.'}
[INFO] Tool response: {'role': 'tool', 'content': '{"height": 187, "weight": 99, "bmi": "Il tuo BMI \\u00e8 28.31, che \\u00e8 considerato Sovrappeso."}', 'tool_call_id': 'call_11y8VnXgKnehg0rTWk4v4ZYa'}
